In [ ]:
import pandas as pd

In [ ]:
file = "data/marc_datenbank_170514.xlsx"

In [ ]:
df = pd.read_excel(file)

In [ ]:
df.fillna(value=False, inplace=True)

In [ ]:
# create concept scheme:
# row['Bombertyp']
plane_type_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='plane_type'
)
# row['Absturzursache']
reason_of_crash_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='reason_of_crash'
)

In [ ]:
df

In [ ]:
for i, row in df.iterrows():
    bomb, _ = Bomber.objects.get_or_create(
        macr_nr=row['MACR-Nr']
    )
    sqr_wrt_name = "{}__{}__{}".format(
        row['Air Force'], row['Bomber Group'], row['Squadron']
    )
    sqd, _ = Institution.objects.get_or_create(
        legacy_id=sqr_wrt_name
    )
    sqd.written_name = "{} {} {}".format(
        row['Air Force'], row['Bomber Group'], row['Squadron']
    )
    sqd.save()
    bomb.squadron = sqd
    
    if row['Angriffsziel'] != 'False':
        target_place, _ = Place.objects.get_or_create(
            name=row['Angriffsziel']
        )
        bomb.target_place = target_place
    
    if row['Sichtung-Ort'] != 'False':
        last_seen, _ = Place.objects.get_or_create(
            name=row['Sichtung-Ort']
        )
        bomb.last_seen = last_seen

    if row['Absturzort'] != 'False':
        crash_place, _ = Place.objects.get_or_create(
            name=row['Absturzort']
        )
        bomb.crash_place = crash_place
    
    if row['Bombertyp'] != 'False':
        plane_type, _ = SkosConcept.objects.get_or_create(
            pref_label=row['Bombertyp']
        )
        plane_type.scheme.set([plane_type_scheme])
        bomb.plane_type = plane_type
    
    if row['Bombername'] != 'False':
        bomb.name = row['Bombername']
    
    if row['Absturzursache'] != 'False':
        reason_of_crash, _ = SkosConcept.objects.get_or_create(
            pref_label=row['Absturzursache']
        )
        reason_of_crash.scheme.set([reason_of_crash_scheme])
        bomb.reason_of_crash = reason_of_crash
     
    bomb.save()

In [ ]:
IdProvider.objects.all().delete()